In [1]:
from system import *
s1 = system('T',MMA=True)

Initializing system T  ...
Constructing states and excitations...
Constructing ground and first-excited statespace...


array(['0g000000000', '0g000000010', '0g000010000', '0g000010010',
       '0g010000000', '0g010000010', '0g010010000', '0g010010010'],
      dtype='<U44')

Constructing gs_hamiltonian ...
Constructing e1_hamiltonian ...
Constructing interactions V_plus and V_minus ...
Constructing NJ_hamiltonian  ...
Inverting NJ_hamiltonian  ...
Constructing eff_hamiltonian and effective lindblau operators ...

System  T  initialized in 166.0 seconds.


# Effective Hamiltonian as Real part

Will output $Q_n$ from expression:

$$\Delta_n = - \dfrac{\Omega^2}{\gamma} \text{Re}\{Q_n \} $$

In [5]:
diag = 2  #input n corresponding to diag element

var('gamma','DE','De','g','g_f','Omega','v','gamma_f','gamma_g',domain='positive')
var('DEt',domain='positive',  latex_name =r'\tilde{\Delta }_E')
var('Det',domain='positive',  latex_name =r'\tilde{\Delta }_e')
var('DEg',domain='positive',  latex_name =r'\Delta_{E\gamma}')
var('Deg',domain='positive',  latex_name =r'\Delta_{e\gamma}')
var('kappa_c','kappa_b',domain='positive')
var('C',domain='positive')
var('c_k',domain='positive',  latex_name =r'c_\kappa')

#(gamma_f=gamma)#gamma_g=0,

#Take only one of the compl conj terms. Switch maybe second 0 to 1
min_expr_len = 10**10
for term in range(2):
    b = s1.eff_hamiltonian_gs[diag,diag]
    b = b.operands()[1].operands()[term]  #  + b.operands()[1].operands()[3] use if obtained with sage

    b = b.subs(DE=+0.5*I*gamma_f + 0.5*I*gamma_g + DEt*gamma)
    b = b.subs(De=+0.5*I*gamma   +Det*gamma)
    #var('n')
    #b = b.subs(DE=DEg*gamma)
    #b = b.subs(De=Deg*gamma)
    
    var('R_f',domain='real')#ratio  (g_f/g)^2
    b = b.subs(g_f=g*sqrt(R_f))

    var('R_v',domain='real',  latex_name =r'R_{\nu}') #ratio (v/g)^2
    b = b.subs(v=g*sqrt(R_v))

    var('r_b',domain='real')
    b = b.subs(kappa_b=gamma/r_b)


    var('r_g',domain='real',latex_name =r'r_g')
    b = b.subs(gamma_g=gamma*r_g)


    var('r_f',domain='real',latex_name =r'r_f')
    b = b.subs(gamma_f=gamma*r_f)


    b = b.subs(g = sqrt(C*kappa_c*gamma)) 
    
    b = b.subs(r_b = c_k/C)
    
    b = b*gamma  #Omega has to be substituted with Omega/gamma
    

    b = b._mathematica_().Factor()._sage_()
    b = MMA_simplify(b,full=True)
    #holds the smallest expression
    if len(str(b))<min_expr_len:
        min_expr_len = len(str(b))
        h_eff = b

show( symround(h_eff))

((((2*I*C - 4*Det)*R_v*c_k + 2.5*I*C - 2.2*Det)*R_v*c_k + 0.62*I*C - 0.38*Det)*R_v*c_k + 0.031*I*C - 0.016*Det)/((((2*I*Det*R_v*c_k + C + 1.5*I*Det)*R_v*c_k + 0.50*C + 0.38*I*Det)*R_v*c_k + 0.062*C + 0.031*I*Det)*C*R_f + ((((2*I*C - 4*Det)*R_v*c_k + 2.5*I*C - 2.2*Det)*R_v*c_k + 0.62*I*C - 0.38*Det)*R_v*c_k + 0.031*I*C - 0.016*Det)*DEt)

# Expand linearly in the limit:   $C\rightarrow \infty$

In [61]:
#with mathematica
q = b.subs(C=1/x)
q = MMA_simplify(q)
tayl = q._mathematica_().Series((x,0,1)).Normal()._sage_()
tayl = tayl.subs(x=1/C)
#t = q._mathematica_().Series((x,0,2))
#t_exp = t[3]
#t_st = t[4]
#tayl = 0
#for i in range(1,len(t_exp)):
#    tayl += MMA_simplify( t_exp[i]._sage_() ) / C**(i-1+Integer(t[4]))

show(symround(tayl))

#with sage
#tayl = b.taylor(C_c,infinity, 1)
#show(symround(MMA_simplify(tayl)))

((2*I*R_v*c_k + 0.66*I)*R_v*c_k + 0.042*I)/(((2*I*DE + 1.3*R_f)*R_v^2*c_k^2 + (0.66*I*DE + R_f)*R_v*c_k + 0.042*I*DE + 0.083*R_f)*C)

# Lindblau Operators

Choose which of the Lindblau operators to view and which non zero element of it

In [13]:
def lind(lind_op,which):    
    L_matrix_orig = s1.lindblau_list[lind_op]
    L_matrix      = s1.eff_lindblau_list[lind_op] #lindblau_list[lind_op] ##(gamma_g=0,gamma_f=gamma)

    L_nonzeros = []
    L_nonzeros_pos = []
    #find non zero elements
    for i in  range(L_matrix.nrows()):
        for j in  range(L_matrix.ncols()):
            if not L_matrix[i,j].is_zero():
                L_nonzeros.append(L_matrix[i,j])
                L_nonzeros_pos.append((i,j))

    #show what these elements do 
    #for t in L_nonzeros_pos:
    #    print(f' |{s1.gs_e1_dec_states[t[0]]}> < {s1.gs_e1_dec_states[t[1]]} | ')

    L_elem = L_nonzeros[which]

    var('gamma_f')
    #L_elem = L_elem.subs(DE=+0.5*I*gamma_f + 0.5*I*gamma_g + DEt*gamma)
    #L_elem = L_elem.subs(De=+0.5*I*gamma   +Det*gamma)
    L_elem = L_elem.subs(DE=gamma*DEg)
    L_elem = L_elem.subs(De=gamma*Deg)                     
    #COUPLINGS
    var('R_f',domain='real')#ratio  (g_f/g)^2
    L_elem = L_elem.subs(g_f=g*sqrt(R_f))

    var('R_v',domain='real',  latex_name =r'R_{\nu}') #ratio (v/g)^2
    L_elem = L_elem.subs(v=g*sqrt(R_v))

    #ABSORBTIONS
    var('r_b',domain='real')
    L_elem = L_elem.subs(kappa_b=gamma/r_b)

    var('gamma_g')
    var('r_g',domain='real',latex_name =r'r_g')
    L_elem = L_elem.subs(gamma_g=gamma*r_g)

    var('gamma_f')
    var('r_f',domain='real',latex_name =r'r_f')
    L_elem = L_elem.subs(gamma_f=gamma*r_f)

    L_elem = L_elem.subs(g = sqrt(C*gamma*kappa_c)) 

    L_elem = L_elem.subs(r_b = c_k/C)
    
    L_elem = L_elem._mathematica_().Factor()._sage_()


    L_elem = MMA_simplify(L_elem,full=True)

    L_elem = SR(str(L_elem).replace('Sqrt','sqrt'))

    

    show( symround(L_elem))

    print(f' |{s1.gs_e1_dec_states[L_nonzeros_pos[which][0]]}> < {s1.gs_e1_dec_states[L_nonzeros_pos[which][1]]} | \n')
    print(f' Original coefficient :')
    show(s1.L_coeffs[lind_op])
    return L_elem

lind_op = 1  #which lindblau operator
which   = 1  #which non zero element to access
L_elem = lind(lind_op,which)

(2*I*((C + 1.5*I*Deg + 0.75)*R_v*c_k + 0.75*C + 0.50*I*Deg + 0.25)*R_v*c_k + 0.12*I*C - 0.062*Deg + 0.031*I)*sqrt(gamma*r_g)*Omega/(((C*(2*I*Deg + 1)*R_f + (2*I*C - 3*Deg + 1.5*I)*DEg + (C + 1.5*I*Deg + 0.75)*r_f + (C + 1.5*I*Deg + 0.75)*r_g)*R_v^2*c_k^2 + ((2*C + 1.5*I*Deg + 0.75)*C*R_f + (1.5*I*C - Deg + 0.50*I)*DEg + (0.75*C + 0.50*I*Deg + 0.25)*r_f + (0.75*C + 0.50*I*Deg + 0.25)*r_g)*R_v*c_k + (0.25*C*R_f + 0.12*I*DEg + 0.062*r_g)*(C + 0.50*I*Deg + 0.25) + (0.062*C + 0.031*I*Deg + 0.016)*r_f)*gamma)

 |0f000001> < 0g000001 | 

 Original coefficient :


sqrt(gamma_g)

# Expand linearly in the limit:   $C\rightarrow \infty$

In [14]:
#with mathematica

q = L_elem.subs(C=1/x)
q = MMA_simplify(q)
t = q._mathematica_().Series((x,0,1)).Normal()

tayl = t._sage_().subs(x=1/C)
tayl = SR(str(tayl).replace('Sqrt','sqrt'))
show(symround(tayl))

((2*I*R_v*c_k + 1.5*I)*R_v*c_k + 0.12*I)*sqrt(gamma*r_g)*Omega/((2*R_f*R_v*c_k + 0.25*R_f)*C*gamma)

((8.00000000000000*Det*R_f + 16.0000000000000*DEt)*C^2*r_c^2 + 1.00000000000000*DEt*Det^2)/((4.00000000000000*Det^2*R_f^2 + 16.0000000000000*DEt*Det*R_f + 16.0000000000000*DEt^2)*C^2*r_c^2 + 1.00000000000000*DEt^2*Det^2)

In [2]:
s1.gs_states

NameError: name 's1' is not defined